# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [43]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [44]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [45]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')] # gather any links reffered to on this page
        self.links = [link for link in links if link] # collected links

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [46]:
ed = Website("https://quiseol.com")
ed.links

['/entry/LLM-comparison',
 '/entry/generators-yield',
 '/entry/tistory-hELLO-skin',
 '/entry/taobao-large-cargo-2',
 '/entry/WinError-10014',
 '/?page=1',
 '/?page=2',
 '/?page=3',
 '/?page=4',
 '/?page=19',
 '/?page=2',
 'https://quiseol.com/',
 '/category',
 '/category/IT%2C%20Computer',
 '/category/%EA%B7%B8%20%EC%99%B8%20%EA%B2%BD%ED%97%98%EA%B8%B0',
 'https://www.instagram.com/seoquiseol',
 '/notice',
 '/entry/esim',
 '/entry/customs-import-declaration',
 '/entry/taobao-large-cargo-1',
 '/entry/shillahotel-s-membership-brown',
 '/entry/china-stopover-2',
 '/entry/LLM-comparison',
 '/entry/generators-yield',
 '/entry/tistory-hELLO-skin',
 '/entry/taobao-large-cargo-2',
 '/entry/WinError-10014',
 'https://pronist.tistory.com/5',
 'https://github.com/pronist',
 '#hELLO']

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [47]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [48]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [49]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [50]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://quiseol.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/entry/LLM-comparison
/entry/generators-yield
/entry/tistory-hELLO-skin
/entry/taobao-large-cargo-2
/entry/WinError-10014
/?page=1
/?page=2
/?page=3
/?page=4
/?page=19
/?page=2
https://quiseol.com/
/category
/category/IT%2C%20Computer
/category/%EA%B7%B8%20%EC%99%B8%20%EA%B2%BD%ED%97%98%EA%B8%B0
https://www.instagram.com/seoquiseol
/notice
/entry/esim
/entry/customs-import-declaration
/entry/taobao-large-cargo-1
/entry/shillahotel-s-membership-brown
/entry/china-stopover-2
/entry/LLM-comparison
/entry/generators-yield
/entry/tistory-hELLO-skin
/entry/taobao-large-cargo-2
/entry/WinError-10014
https://pronist.tistory.com/5
https://github.com/pronist
#hELLO


In [51]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"} #claude는 필요업슴
    )
    result = response.choices[0].message.content #choices는 다양한 variation이 있는데 그 중 하나 고른거임
    return json.loads(result)

In [52]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://quiseol.com")
huggingface.links

['/entry/LLM-comparison',
 '/entry/generators-yield',
 '/entry/tistory-hELLO-skin',
 '/entry/taobao-large-cargo-2',
 '/entry/WinError-10014',
 '/?page=1',
 '/?page=2',
 '/?page=3',
 '/?page=4',
 '/?page=19',
 '/?page=2',
 'https://quiseol.com/',
 '/category',
 '/category/IT%2C%20Computer',
 '/category/%EA%B7%B8%20%EC%99%B8%20%EA%B2%BD%ED%97%98%EA%B8%B0',
 'https://www.instagram.com/seoquiseol',
 '/notice',
 '/entry/esim',
 '/entry/customs-import-declaration',
 '/entry/taobao-large-cargo-1',
 '/entry/shillahotel-s-membership-brown',
 '/entry/china-stopover-2',
 '/entry/LLM-comparison',
 '/entry/generators-yield',
 '/entry/tistory-hELLO-skin',
 '/entry/taobao-large-cargo-2',
 '/entry/WinError-10014',
 'https://pronist.tistory.com/5',
 'https://github.com/pronist',
 '#hELLO']

In [54]:
get_links("https://quiseol.com")

{'links': [{'type': 'company homepage', 'url': 'https://quiseol.com/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [55]:
def get_all_details(url): #url가져온 다음 gather information
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url) #makes a call out to GPT
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [56]:
print(get_all_details("https://quiseol.com"))

Found links: {'links': [{'type': 'homepage', 'url': 'https://quiseol.com/'}]}
Landing page:
Webpage Title:
QUISEOL — QUISEOL
Webpage Contents:
LLM 모델별(gpt, gemini, claude 등) 성능 비교하는 법
2025.05.01
·
IT, Computer
서론LLM 사용에 있어서 성능의 차이가 수치적으로 궁금할 수 있다. 아무래도 체감하는 것과 객관적인 지표는 다를 수 있으니까 말이다. 그런 의미에서 이번 포스팅에서는 LLM 모델을 모델별 성능 비교한 사이트에 들어가고 그 의미를 알아보도록 하겠다. 아래 사이트를 참고하여, 하나하나 그 의미를 알아보자. LLM Leaderboard 2025This AI leaderboard shows comparison of capabilities, price and context window for leading commercial and open-source LLMs, based on the benchmark data provided in technical reports in 2025.www.vellum.ai Rea..
파이썬 제너레이터(generator), yield Python
2025.04.30
·
IT, Computer
서론파이썬에서는 yield 키워드를 통해 iterate하는 걸 볼 수 있다. 한 번에 하나의 요소를 통해 루프할 수 있는 방법이다. 특히 모든 것을 한 번에 메모리에 로드하고 싶지 않은 경우 한 번에 하나씩 데이터를 쉽게 처리할 수 있도록 하는 것이 목적이다. 쉽게 말해, 모든 데이터를 훑지 않고, 사용자가 원할 때 하나씩 값을 처리하는 것이다. 이렇게 값을 처리하는 방법을 Lazy Evaluation이라고 한다. 한 번에 하나씩 하는게 뭔말인지 아래 예제를 통해서 알아보자.yield 란?내가 아는 yield는 일본드라마인데(허허) 싶겠지만, 예제에 앞서 yield 키워드를 알아야 

In [61]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt =  "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [62]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [65]:
get_brochure_user_prompt("HuggingFace", "https://quiseol.com")

Found links: {'links': [{'type': 'homepage', 'url': 'https://quiseol.com/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nQUISEOL — QUISEOL\nWebpage Contents:\nLLM 모델별(gpt, gemini, claude 등) 성능 비교하는 법\n2025.05.01\n·\nIT, Computer\n서론LLM 사용에 있어서 성능의 차이가 수치적으로 궁금할 수 있다. 아무래도 체감하는 것과 객관적인 지표는 다를 수 있으니까 말이다. 그런 의미에서 이번 포스팅에서는 LLM 모델을 모델별 성능 비교한 사이트에 들어가고 그 의미를 알아보도록 하겠다. 아래 사이트를 참고하여, 하나하나 그 의미를 알아보자. LLM Leaderboard 2025This AI leaderboard shows comparison of capabilities, price and context window for leading commercial and open-source LLMs, based on the benchmark data provided in technical reports in 2025.www.vellum.ai Rea..\n파이썬 제너레이터(generator), yield Python\n2025.04.30\n·\nIT, Computer\n서론파이썬에서는 yield 키워드를 통해 iterate하는 걸 볼 수 있다. 한 번에 하나의 요소를 통해 루프할 수 있는 방법이다. 특히 모든 것을 한 번에 메모리에 로드하고 싶지 않은 경우 한 번에 하나씩 데이터를 쉽게 처리할 수 있도록 하는 것이 목적이다. 쉽게 말해, 모든 데이터를 훑지 않고, 사용자가 원할 때 하나씩 값을 처리하는 것이다. 이렇게 

In [66]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [70]:
create_brochure("quiseol", "https://quiseol.com")

Found links: {'links': [{'type': 'home page', 'url': 'https://quiseol.com/'}]}


# QUISEOL Brochure

---

## Welcome to QUISEOL

At **QUISEOL**, we specialize in the performance analysis of cutting-edge LLM (Large Language Model) technologies. With a dedicated focus on the latest models such as GPT, Gemini, and Claude, we provide insightful comparisons to help our users make informed decisions based on their specific needs.

---

## Our Vision

QUISEOL aims to bridge the gap between subjective experiences and objective benchmarks. Our platform ranks LLMs based on capabilities, pricing, and context, providing users with comprehensive data from the latest technical reports. We continually explore the evolving landscape of artificial intelligence, ensuring our insights remain relevant and impactful.

---

## Company Culture

At QUISEOL, we foster a **collaborative and innovative** work environment. Our team is passionate about technology and committed to continuous learning. We encourage creativity and critical thinking, enabling every member to contribute uniquely to our mission. We believe in balancing hard work with a fun atmosphere, promoting both personal and professional growth.

---

## Who We Serve

Our customer base includes:

- **Technologists** looking for in-depth analyses of LLM performance.
- **Businesses** aiming to select the best AI tools for their operations.
- **Developers** seeking guidance on how to implement AI solutions effectively.

We are dedicated to providing valuable resources, practical advice, and expert analyses to ensure our users benefit maximally from our services.

---

## Career Opportunities

QUISEOL is always looking for talented individuals to join our team! If you are passionate about AI, data analysis, or software development, we want to hear from you. We offer:

- A dynamic team environment
- Opportunities for professional development
- Competitive compensation and benefits

Explore your potential and grow with us at QUISEOL.

---

Join us in shaping the future of AI technology. For more information, visit us at **[Quiseol.com](http://quiseol.com)** or connect with us on our social media platforms.

---

**QUISEOL—Where Innovation Meets Analysis**

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [71]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [72]:
stream_brochure("Quiseol", "https://quiseol.com")

Found links: {'links': [{'type': 'homepage', 'url': 'https://quiseol.com/'}, {'type': 'about page', 'url': 'https://quiseol.com/entry/LLM-comparison'}, {'type': 'about page', 'url': 'https://quiseol.com/entry/generators-yield'}]}



# Welcome to Quiseol

Quiseol is your go-to destination for understanding the dynamic performance of Large Language Models (LLMs) such as GPT, Gemini, and Claude. Our commitment is to provide you with comprehensive insights and resources tailored for developers, IT professionals, and individuals keen on leveraging AI technologies.

## What We Do

At Quiseol, we focus on:

- **Performance Comparison**: We analyze and publish detailed comparisons of various LLMs, providing insightful metrics on capabilities, pricing, and context windows using the latest benchmark data.
- **Educational Content**: We create informative articles on topics that matter to our community, such as Python programming techniques and effective data handling methods.
- **User-Centric Resources**: Our blog features tutorials and guides that help users troubleshoot common issues, as well as advice on best practices in technology use.

## Our Customers

Our primary audience includes:

- **IT Professionals**: Developers and tech enthusiasts looking to optimize their use of LLMs.
- **Businesses**: Organizations seeking to implement AI solutions to enhance their operations.
- **Students & Learners**: Individuals eager to expand their knowledge in technology and programming.

## Company Culture

Quiseol thrives on innovation, collaboration, and inclusivity. We believe in fostering a work environment where ideas can flourish, and every team member's voice is valued. Our culture encourages continuous learning, and we take pride in supporting professional growth through ongoing training and development opportunities.

## Careers at Quiseol

Join us at Quiseol and be a part of a forward-thinking team that is shaping the future of AI technology. We are constantly on the lookout for talented individuals who are passionate about technology and eager to make a difference.

**Current Openings:**
- Software Engineer
- Data Scientist
- Content Creator

Explore our career opportunities and be a part of the technological revolution!

## Get in Touch

Have questions or want to learn more? Visit us at [quiseol.com](https://www.quiseol.com) and connect with our community today!

**Follow us on social media for the latest updates!**

---

*Quiseol - Empowering Technology Through Knowledge and Innovation*



In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>